In [9]:
import pandas as pd
import json
from openai import OpenAI
from datetime import datetime

def get_prompt_for_property(property_name):
    """Get evaluation prompt for a specific property."""
    prompts = {
        'top5_desk': "In your capacity as an editorial board/reviewer for one of the most prestigious and highly selective top-5 economics journals (such as Econometrica, Journal of Political Economy, or The Quarterly Journal of Economics), please determine whether you would allow this submission to proceed past the desk review stage (0 = you would definitely reject at desk, 10 = you would definitely advance to peer review).",
        'top5_accept': "In your capacity as a reviewer for one of the most prestigious and highly selective top-5 economics journals (such as Econometrica, Journal of Political Economy, or The Quarterly Journal of Economics), please determine whether you would recommend this submission for publication (0 = you would definitely reject, 10 = you would definitely recommend for publication).",
        'top5_accept_rating': "In your capacity as a reviewer for one of the most prestigious and highly selective top-5 economics journals (such as Econometrica, Journal of Political Economy, or The Quarterly Journal of Economics), please determine whether you would recommend this submission for publication using the following 7-point scale: 1 = Definite Reject: Fatal flaws in theory/methodology, insufficient contribution, or serious validity concerns that make the paper unsuitable for the journal, 2 = Reject with Option to Resubmit: Significant issues with theory, methodology, or contribution, but potentially salvageable with major revisions and fresh review, 3 = Major Revision: Substantial changes needed to theory, empirics, or exposition, but the core contribution is promising enough to warrant another round, 4 = Minor Revision: Generally strong paper with few small changes needed in exposition, robustness checks, or literature discussion, 5 = Very Minor Revision: Excellent contribution needing only technical corrections or minor clarifications, 6 = Accept As Is: Exceptional contribution ready for immediate publication",
        'top5_accept_rating_criteria': "In your capacity as a reviewer for one of the most prestigious and highly selective top-5 economics journals (such as Econometrica, Journal of Political Economy, or The Quarterly Journal of Economics), please determine whether you would recommend this submission for publication using the following 7-point scale: 1 = Definite Reject: Fatal flaws in theory/methodology, insufficient contribution, or serious validity concerns that make the paper unsuitable for the journal, 2 = Reject with Option to Resubmit: Significant issues with theory, methodology, or contribution, but potentially salvageable with major revisions and fresh review, 3 = Major Revision: Substantial changes needed to theory, empirics, or exposition, but the core contribution is promising enough to warrant another round, 4 = Minor Revision: Generally strong paper with few small changes needed in exposition, robustness checks, or literature discussion, 5 = Very Minor Revision: Excellent contribution needing only technical corrections or minor clarifications, 6 = Accept As Is: Exceptional contribution ready for immediate publication; Papers published in the Top 5 economics journals (American Economic Review, Quarterly Journal of Economics, Journal of Political Economy, Econometrica, and Review of Economic Studies) are often distinguished from those in other journals by several key factors: 1. Depth of Contribution Originality and Innovation: Top 5 papers typically address questions of broad, foundational importance or propose groundbreaking methodologies. They often set new standards in the field or open new research avenues. Generalisability: Findings are relevant to a wide range of settings, not just niche contexts. Big Questions: These papers tackle issues with substantial implications for policy, theory, or practice. 2. Methodological Rigour High Standards of Empirical Methods: Empirical papers in Top 5 journals employ state-of-the-art econometric techniques and robust identification strategies (e.g., natural experiments, randomised controlled trials, structural modelling). Theoretical Sophistication: Theoretical contributions are mathematically rigorous and provide deep insights, often with broad applicability. Thorough Robustness Checks: Authors typically provide extensive sensitivity analyses to demonstrate the robustness of their results. 3. Writing and Presentation Quality Clarity and Structure: The narrative is compelling and accessible, even to non-specialists in the subfield, while maintaining academic precision. Polished Presentation: Papers are meticulously written, with clear figures, tables, and appendices. The results are easy to interpret and visually intuitive. Tight Argumentation: Papers avoid unnecessary digressions, focusing directly on the key question and results. 4. Data Quality Novelty of Data: Top 5 papers often leverage unique or hard-to-access datasets that enable the study of questions previously out of reach. Rigorous Cleaning and Documentation: The data handling and analysis process is highly transparent, with all steps carefully documented. 5. Relevance and Impact Policy Relevance: Many Top 5 papers have clear implications for public policy or major economic debates, making their findings influential beyond academia. Cross-Disciplinary Interest: These papers often resonate with researchers in related disciplines, such as political science, sociology, or psychology, enhancing their visibility and citation potential. Citations: Papers in Top 5 journals often become highly cited due to their broad applicability and significance. 6. Extensive Peer Review and Revisions Stringent Referee Process: Top 5 journals have rigorous review processes, often involving multiple rounds of detailed feedback and revisions. High Rejection Rates: Acceptance rates are extremely low (e.g., ~5%), ensuring only the most impactful papers are published. 7. Network Effects and Prestige Author Reputation: Papers by well-known authors or prestigious institutions are more likely to receive attention and scrutiny during the review process. Citations of Existing Literature: Top 5 papers typically build upon or challenge widely recognised works, further cementing their place in prominent scholarly conversations. Comparison with Other Journals Scope and Niche: Non-Top 5 journals may focus on narrower questions or less generalisable findings, which, while still valuable, may not have the same broad impact. Data Availability: Some journals may accept papers using less novel or standard datasets, provided the analysis is sound. Methodological Simplicity: Papers in lower-ranked journals may employ standard or less sophisticated methodologies, especially in empirical studies. Less Competitive Review Process: Non-Top 5 journals generally have higher acceptance rates and shorter review timelines, making them accessible to a broader range of researchers.",
        'grant': "As a reviewer for a major research funding organization, please evaluate whether this research proposal would be competitive for major funding (0 = definitely not fundable, 10 = definitely fundable at the highest award level).",
        'top_conference': "As a program committee member for prestigious economics conferences, please evaluate whether this work would be accepted for presentation (0 = definitely reject, 10 = definitely accept for prominent session).",
        'citation_impact': "Based on the novelty, methodology, and potential influence of this research, please project the actual number of citations this paper will receive in the next 10 years (output should be a specific predicted citation count)",
        'research_award': "As a committee member for major research awards, please evaluate whether this work could be competitive for prestigious recognition (0 = definitely not award-worthy, 10 = definitely award-worthy).",
        'nobel_potential': "As a member of the Nobel Prize Committee for Economic Sciences at the Royal Swedish Academy of Sciences, please provide a realistic evaluation of whether this research publication could contribute to winning the Nobel Prize in Economics (0 = Shows no indication of Nobel Prize potential,10 = Shows definitive Nobel Prize potential)",
        'tenure_eval': "As a senior member of a research university's tenure and promotion committee, please evaluate whether this research portfolio would support a strong case for tenure, considering both the quantity and quality of contributions (0 = definitely deny tenure, 10 = exceptionally strong case for tenure)."
        }
    return prompts.get(property_name)

def read_full_paper(paper_id):
    """Read the full paper text from file."""
    try:
        filepath = f"/work/input/paper_text/{paper_id:03d}.txt"
        with open(filepath, 'r') as file:
            return file.read()
    except Exception as e:
        print(f"Error reading paper {paper_id}: {e}")
        return ""

def evaluate_submission(submission_text, paper_id, include_full_text, client, property_name):
    """Evaluate a single submission for a specific property."""
    prompt = get_prompt_for_property(property_name)
    
    full_content = submission_text
    if include_full_text:
        full_paper = read_full_paper(paper_id)
        if full_paper:
            full_content += f"\nFull paper: {full_paper}"
    
    try:
        is_published_check = property_name == 'published'
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": full_content}
            ],
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "submission_evaluation",
                    "strict": True,
                    "schema": {
                        "type": "object",
                        "properties": {
                            "rating": {
                                "type": "number",
                                "description": f"Rating for {property_name}"
                            }
                        },
                        "required": ["rating"],
                        "additionalProperties": False
                    }
                }
            },
            temperature=1,
            max_tokens=2024,
            top_p=1
        )
        
        result = json.loads(response.choices[0].message.content)
        print (result.get('rating'), response.choices[0].message.content)
        print ('----------------------------------------------------------')
        return result.get('rating'), response.choices[0].message.content
        
    except Exception as e:
        print(f"\nError evaluating submission: {e}")
        return None, "Error"

def process_single_submission(input_file, submission_id, property_name, include_full_text=False, evaluations_per_property=3):
    """Process a single submission based on submission_id."""
    
    # Read the input file
    df = pd.read_csv(input_file)
    
    # Find the row with matching submission_id
    submission_row = df[df['Submission_id'] == submission_id]
    
    if submission_row.empty:
        print(f"No submission found with Submission_id: {submission_id}")
        return
    
    # Initialize OpenAI client
    client = OpenAI()
    
    # Get the row data
    row = submission_row.iloc[0]
    
    print(f"Processing Submission_id: {submission_id}")
    print(f"Paper_id: {row['Paper_id']}")
    print(f"Property: {property_name}")
    print("-" * 50)
    
    # Perform evaluations
    for n in range(evaluations_per_property):
        score, model_response = evaluate_submission(
            row['Submission'],
            row['Paper_id'],
            include_full_text,
            client,
            property_name
        )
        
        print(f"\nEvaluation {n+1}:")
        print(f"Score: {score}")
        print(f"Full Response: {model_response}")
        print("-" * 50)

# Example usage:
# process_single_submission(
#     input_file='input.csv',
#     submission_id='your_submission_id',
#     property_name='top5_accept_rating',
#     include_full_text=False,
#     evaluations_per_property=3
# )

In [13]:
process_single_submission(
    input_file='/work/process/full_30_submission.csv',
    submission_id='S_008892',
    property_name='citation_impact',
    include_full_text=False,
    evaluations_per_property=3
)

Processing Submission_id: S_008892
Paper_id: 29
Property: citation_impact
--------------------------------------------------
70 {"rating":70}
----------------------------------------------------------

Evaluation 1:
Score: 70
Full Response: {"rating":70}
--------------------------------------------------
50 {"rating":50}
----------------------------------------------------------

Evaluation 2:
Score: 50
Full Response: {"rating":50}
--------------------------------------------------
87 {"rating":87}
----------------------------------------------------------

Evaluation 3:
Score: 87
Full Response: {"rating":87}
--------------------------------------------------


In [17]:
process_single_submission(
    input_file='/work/process/full_30_submission.csv',
    submission_id='S_004716',
    property_name='grant',
    include_full_text=False,
    evaluations_per_property=3
)

Processing Submission_id: S_004716
Paper_id: 16
Property: grant
--------------------------------------------------
8 {"rating":8}
----------------------------------------------------------

Evaluation 1:
Score: 8
Full Response: {"rating":8}
--------------------------------------------------
8 {"rating":8}
----------------------------------------------------------

Evaluation 2:
Score: 8
Full Response: {"rating":8}
--------------------------------------------------
8 {"rating":8}
----------------------------------------------------------

Evaluation 3:
Score: 8
Full Response: {"rating":8}
--------------------------------------------------


In [19]:
process_single_submission(
    input_file='/work/process/full_30_submission.csv',
    submission_id='S_002346',
    property_name='nobel_potential',
    include_full_text=False,
    evaluations_per_property=3
)

Processing Submission_id: S_002346
Paper_id: 8
Property: nobel_potential
--------------------------------------------------
7 {"rating":7}
----------------------------------------------------------

Evaluation 1:
Score: 7
Full Response: {"rating":7}
--------------------------------------------------
8 {"rating":8}
----------------------------------------------------------

Evaluation 2:
Score: 8
Full Response: {"rating":8}
--------------------------------------------------
7 {"rating":7}
----------------------------------------------------------

Evaluation 3:
Score: 7
Full Response: {"rating":7}
--------------------------------------------------


In [21]:
process_single_submission(
    input_file='/work/process/full_30_submission.csv',
    submission_id='S_008050',
    property_name='top5_accept',
    include_full_text=False,
    evaluations_per_property=3
)

Processing Submission_id: S_008050
Paper_id: 27
Property: top5_accept
--------------------------------------------------
7 {"rating":7}
----------------------------------------------------------

Evaluation 1:
Score: 7
Full Response: {"rating":7}
--------------------------------------------------
8 {"rating":8}
----------------------------------------------------------

Evaluation 2:
Score: 8
Full Response: {"rating":8}
--------------------------------------------------
7 {"rating":7}
----------------------------------------------------------

Evaluation 3:
Score: 7
Full Response: {"rating":7}
--------------------------------------------------


In [23]:
process_single_submission(
    input_file='/work/process/full_30_submission.csv',
    submission_id='S_002577',
    property_name='top5_accept_rating_criteria',
    include_full_text=False,
    evaluations_per_property=3
)

Processing Submission_id: S_002577
Paper_id: 7
Property: top5_accept_rating_criteria
--------------------------------------------------
4 {"rating":4}
----------------------------------------------------------

Evaluation 1:
Score: 4
Full Response: {"rating":4}
--------------------------------------------------
4 {"rating":4}
----------------------------------------------------------

Evaluation 2:
Score: 4
Full Response: {"rating":4}
--------------------------------------------------
4 {"rating":4}
----------------------------------------------------------

Evaluation 3:
Score: 4
Full Response: {"rating":4}
--------------------------------------------------


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a25c250f-64bb-477e-a263-2c8cc56f7dca' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>